# Coding Attention Mechanism

### Simple self-attention mechanism

In [3]:
import torch

inputs = torch.tensor([
    [0.43, 0.15, 0.89], # Your    (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts  (x^3)
    [0.22, 0.58, 0.33], # with    (x^4)
    [0.77, 0.25, 0.10], # one     (x^5)
    [0.05, 0.80, 0.55], # step    (x^6)
])

In [4]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [5]:
query = inputs[1]

atten_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    atten_scores_2[i] = torch.dot(x_i, input_query)

atten_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [6]:
atten_weights_2_temp = atten_scores_2 / atten_scores_2.sum()

In [7]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

softmax_naive(atten_scores_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [8]:
atten_weights_2 = torch.softmax(atten_scores_2, dim=0)

In [9]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += atten_weights_2[i] * inputs[i]

context_vec_2

tensor([0.4419, 0.6515, 0.5683])

### A simple self-attention mechanism without trainable weights

In [10]:
atten_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        atten_scores[i, j] = torch.dot(x_i, x_j)


atten_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [11]:
atten_scores = inputs @ inputs.T
atten_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [12]:
atten_weights = torch.softmax(atten_scores, dim=1)
atten_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [13]:
all_context_vecs = atten_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

### Implementing self-attention mechanism with trainable weights

In [14]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [15]:
torch.manual_seed(123)

w_query = torch.nn.Parameter(torch.rand(d_in, d_out))
w_key = torch.nn.Parameter(torch.rand(d_in, d_out))
w_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [16]:
query_2 = x_2 @ w_query
query_2 

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [17]:
keys = inputs @ w_key
value = inputs @ w_value

In [18]:
atten_score_2 = query_2 @ keys.T
atten_score_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [19]:
d_k = keys.shape[1]
atten_weights_2 = torch.softmax(atten_score_2 / d_k**0.5, dim=-1)
atten_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)

In [20]:
context_vec_2 = atten_weights_2 @ value
context_vec_2

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)

### Implementing self-attention

In [21]:
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.w_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.w_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.w_value = torch.nn.Parameter(torch.rand(d_in, d_out))


    def forward(self, x):
        queries = inputs @ self.w_query
        keys = inputs @ self.w_key
        values = inputs @ self.w_value
        atten_scores = queries @ keys.T
        atten_weights = torch.softmax(atten_scores / d_k**0.5, dim=-1)
        context_vec = atten_weights @ values
        return context_vec


torch.manual_seed(123)
sa_v1 = SelfAttention(d_in, d_out)
sa_v1(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

In [22]:
import torch.nn as nn

class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.w_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)


    def forward(self, x):
        queries = self.w_query(inputs)
        keys = self.w_key(inputs)
        values = self.w_value(inputs)
        atten_scores = queries @ keys.T
        atten_weights = torch.softmax(atten_scores / d_k**0.5, dim=-1)
        context_vec = atten_weights @ values
        return context_vec


torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

### Applying casual attention mask

In [23]:
queries = sa_v2.w_query(inputs)
keys = sa_v2.w_key(inputs)
values = sa_v2.w_value(inputs)

atten_scores = queries @ keys.T
atten_weights = torch.softmax(atten_scores / d_k**2, dim=-1)

In [26]:
context_length = atten_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
masked_simple = atten_weights * mask_simple
masked_simple

tensor([[0.1754, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1793, 0.1666, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1791, 0.1666, 0.1667, 0.0000, 0.0000, 0.0000],
        [0.1736, 0.1667, 0.1668, 0.1633, 0.0000, 0.0000],
        [0.1723, 0.1668, 0.1668, 0.1639, 0.1664, 0.0000],
        [0.1758, 0.1667, 0.1667, 0.1623, 0.1667, 0.1618]],
       grad_fn=<MulBackward0>)

In [27]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
masked_simple_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5183, 0.4817, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3495, 0.3251, 0.3254, 0.0000, 0.0000, 0.0000],
        [0.2590, 0.2487, 0.2488, 0.2435, 0.0000, 0.0000],
        [0.2061, 0.1994, 0.1995, 0.1960, 0.1990, 0.0000],
        [0.1758, 0.1667, 0.1667, 0.1623, 0.1667, 0.1618]],
       grad_fn=<DivBackward0>)

In [ ]:
# Your journey starts with one step

# Your -> journey

mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = atten_scores.masked_fill(mask.bool(), -torch.inf)
masked

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)

In [29]:
atten_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
atten_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

### Masking additional attention weights with dropout 

In [30]:
torch.manual_seed(123)
layer = torch.nn.Dropout(0.5)

layer(atten_weights)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)

### Implementing casual self-attention class

In [31]:
batch = torch.stack((inputs, inputs), dim=0)
batch.shape

torch.Size([2, 6, 3])

In [32]:
import torch.nn as nn

class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.w_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        # to move mask variables to cuda later on, if we are using GPU
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))


    def forward(self, x):
        b, num_tokens, d_in = x.shape
        queries = self.w_query(x)
        keys = self.w_key(x)
        values = self.w_value(x)
        
        atten_scores = queries @ keys.transpose(1, 2)
        atten_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        atten_weights = torch.softmax(
            atten_scores / keys.shape[-1]**0.5, dim=-1
        )
        atten_weights = self.dropout(atten_weights)
        context_vec = atten_weights @ values
        return context_vec


torch.manual_seed(789)
context_length = batch.shape[1]
dropout = 0.0
ca = CasualAttention(d_in, d_out, context_length, dropout)
ca(batch)

tensor([[[-0.0872,  0.0286],
         [-0.0991,  0.0501],
         [-0.0999,  0.0633],
         [-0.0983,  0.0489],
         [-0.0514,  0.1098],
         [-0.0754,  0.0693]],

        [[-0.0872,  0.0286],
         [-0.0991,  0.0501],
         [-0.0999,  0.0633],
         [-0.0983,  0.0489],
         [-0.0514,  0.1098],
         [-0.0754,  0.0693]]], grad_fn=<UnsafeViewBackward0>)

### Multi-head attention

In [34]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False, num_heads=2):
        super().__init__()
        self.heads = nn.ModuleList(
            [CasualAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
            )
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    

torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = batch.shape[-1], 2

mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout=0.0, num_heads=2)
mha(batch)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)

### Mulit-head attention with with weight split

In [61]:
class MulitHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask", torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.w_key(x)
        values = self.w_value(x)
        queries = self.w_query(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)
        queries = queries.transpose(1, 2)

        atten_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        atten_scores.masked_fill_(mask_bool, -torch.inf)
        atten_weights = torch.softmax(atten_scores / keys.shape[-1]**0.5, dim=-1)
        atten_weights = self.dropout(atten_weights)

        # shape -> (b, num_tokens, num_heads, head_dim): (2, 6, 2, 2)
        context_vec = (atten_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec



torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 4
mha = MulitHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]]], grad_fn=<ViewBackward0>)
